In [1]:
from python_profiling.python_profiling_configs import ComposedProfilingConfig
from python_profiling.python_profiling_enums import *
from python_profiling.composed_profiling.composed_profiler import ComposedProfiler

In [2]:
configs = ComposedProfilingConfig(time_profiling_strategy=TimeProfilingStrategy.LINE_TIME_PROFILER,
                                 memory_profiling_strategy=MemoryProfilingStrategy.LINE_MEMORY_PROFILER,
                                 memory_profiling_strategy_params = {'interval': 11})

In [3]:
@ComposedProfiler(composed_profiling_config=configs)
def hello(name):
    print(f'hello {name}')
    
    b = [i for i in range(1000000)]
    1 / 0
    return 1

In [4]:
res = hello(name='bob')
res

hello bob
hello bob


ComposedProfilerResult(time_profiling_result=LineTimeProfilerResult(profiler=type, profiled_func=<function hello at 0x7fda108575b0>), memory_profiling_result=LineMemoryProfilerResult(profiler=LineMemoryProfiler, profiled_func=hello))

In [5]:
print(res)

Time Profiling Result:
Profiler: <class 'python_profiling.time_profiling.line_time_profiler.LineTimeProfiler'>
Profiled Function: hello
Function Kwargs: {'name': 'bob'}
Function Result: None
Function Profiling Result: Timer unit: 1e-09 s

Total time: 0.114862 s
File: /var/folders/k9/6y59tr5d45s1nynfg_sv5zr80000gn/T/ipykernel_32417/3152039017.py
Function: hello at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           @ComposedProfiler(composed_profiling_config=configs)
     2                                           def hello(name):
     3         1     260000.0 260000.0      0.2      print(f'hello {name}')
     4                                               
     5         1  114597000.0    1e+08     99.8      b = [i for i in range(1000000)]
     6         1       5000.0   5000.0      0.0      1 / 0
     7                                               return 1


Function Exceptions: <class 'ZeroDivisionError'>
Memory P

In [6]:
@ComposedProfiler()
def hello(name):
    print(f'hello {name}')
    
    b = [i for i in range(1000000)]
    1 / 0
    return 1

In [7]:
res = hello(name='bob')
print(res)

hello bob
Time Profiling Result:
Profiler: <class 'python_profiling.time_profiling.line_time_profiler.LineTimeProfiler'>
Profiled Function: hello
Function Kwargs: {'name': 'bob'}
Function Result: None
Function Profiling Result: Timer unit: 1e-09 s

Total time: 0.123851 s
File: /var/folders/k9/6y59tr5d45s1nynfg_sv5zr80000gn/T/ipykernel_32417/3949927082.py
Function: hello at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           @ComposedProfiler()
     2                                           def hello(name):
     3         1     812000.0 812000.0      0.7      print(f'hello {name}')
     4                                               
     5         1  123034000.0    1e+08     99.3      b = [i for i in range(1000000)]
     6         1       5000.0   5000.0      0.0      1 / 0
     7                                               return 1


Function Exceptions: <class 'ZeroDivisionError'>
Memory Profiling Result:
Profil

In [8]:
ComposedProfilingConfig.avaliable_profilers()

Profiling type: ProfilingType.TIME_PROFILING
    Profiler: TimeProfilingStrategy.TIME_PROFILER
    Profiler: TimeProfilingStrategy.TIMEIT_PROFILER
    Profiler: TimeProfilingStrategy.LINE_TIME_PROFILER
    Profiler: TimeProfilingStrategy.CALL_GRAPH_TIME_PROFILER

Profiling type: ProfilingType.MEMORY_PROFILING
    Profiler: MemoryProfilingStrategy.PEAK_MEMORY_PROFILER
    Profiler: MemoryProfilingStrategy.OBJECT_ALLOCATION_PROFILER
    Profiler: MemoryProfilingStrategy.LINE_MEMORY_PROFILER

Profiling type: ProfilingType.CALL_GRAPH_PROFILING
    Profiler: 



In [9]:
res.time_profiling_result

LineTimeProfilerResult(profiler=type, profiled_func=<function hello at 0x7fda108576d0>)

In [10]:
res.memory_profiling_result

ObjectAllocationProfilerResult(profiler=ObjectAllocationProfiler, profiled_func=hello)